In [13]:
# %%file outputer.py
from abc import ABC, abstractmethod
from typing import Optional

class Outputer(ABC):
    @abstractmethod
    def output(self, content: str) -> None:
        """输出给定的内容"""
        ...


class PrintOutputer(Outputer):
    def output(self, content: str) -> None:
        print(content)


class FileOutputer(Outputer):
    DEFAULT_PATH = 'product_report.txt'

    def __init__(self, path: Optional[str] = DEFAULT_PATH, **kwargs) -> None:
        self.path = path

    def output(self, content: str) -> None:
        with open(self.path, 'a') as f:
            f.write(f'{content}\n')


class CombineOutputer(Outputer):
    def __init__(self) -> None:
        self._print_outputer = PrintOutputer()
        self._file_outputer = FileOutputer()

    def output(self, content: str) -> None:
        self._print_outputer.output(content)
        self._file_outputer.output(content)

In [14]:
# %%file outputer_factory.py
from typing import Callable, Any

from outputer import Outputer

creation_funcs: dict[str, Callable[..., Outputer]] = {}


def register(kind: str, creation_func: Callable[..., Outputer]) -> None:
    creation_funcs[kind] = creation_func


def unregister(kind: str) -> None:
    creation_funcs.pop(kind, None)


def create(args: dict[str, Any]) -> Outputer:
    the_args = args.copy()
    kind = the_args.pop('kind')
    print(the_args)
    try:
        creation_func = creation_funcs[kind]
        return creation_func(**the_args)
    except KeyError:
        raise ValueError(f'未知的Outputer {kind}') from None

In [16]:
import json
from typing import Optional

from outputer_factory import *


class Product:
    def __init__(self, name: str, price: float) -> None:
        self.name = name
        self.price = price


class ProductReporter:
    NAME_LIST_LENGTH = 60
    PRICE_LENGTH = 5
    COL_NAME = 'NAME'
    COL_PRICE = 'PRICE'
    HEADER_SPLITER_CHARACTER = '_'
    LINE_SPLITER_CHARACTER = '-'

    def __init__(self,
                 products: list[Product],
                 outputer: Outputer,
                 name_list_length: Optional[int] = NAME_LIST_LENGTH,
                 price_length: Optional[int] = PRICE_LENGTH,
                 col_name: Optional[str] = COL_NAME,
                 col_price: Optional[str] = COL_PRICE,
                 header_spliter_character: Optional[str] = HEADER_SPLITER_CHARACTER,
                 line_spliter_character: Optional[str] = LINE_SPLITER_CHARACTER):
        self.products = products
        self.outputer = outputer
        self.name_list_length = name_list_length
        self.price_length = price_length
        self.col_name = col_name
        self.col_price = col_price
        self.header_spliter_character = header_spliter_character
        self.line_spliter_character = line_spliter_character

    def _output(self, content: str) -> None:
        self.outputer.output(content)

    def _get_header(self) -> str:
        return f'{self.col_name: ^{self.name_list_length}}' \
               f'{self.col_price: ^{self.price_length}}'

    def _get_header_spliter(self) -> str:
        size = self.name_list_length + self.price_length
        return self.header_spliter_character * size

    def _output_header(self) -> None:
        # 获取标题内容
        header = self._get_header()
        self._output(header)
        # 获取分割线
        spliter = self._get_header_spliter()
        self._output(spliter)

    def _handle_product_name(self, product: Product) -> str:
        name_length = len(product.name)
        half_length = int(name_length / 3 - 1)

        left_text = product.name[:half_length]
        right_text = product.name[-half_length:]

        name = f'{left_text}..{right_text}' \
            if name_length > self.name_list_length else product.name
        return name

    def _handle_product_price(self, product: Product) -> str:
        return str(product.price)

    def _get_line_item(self, name: str, price: str) -> str:
        return f'{name: <{self.name_list_length}}{price: >{self.price_length}}'

    def _get_line_spliter(self) -> str:
        size = self.name_list_length + self.price_length
        return self.line_spliter_character * size

    def _output_line_item(self, name: str, price: str) -> None:
        # 1.获取每行的输出内容
        line_item = self._get_line_item(name, price)
        self._output(line_item)
        # 2.获取分隔符
        spliter = self._get_line_spliter()
        self._output(spliter)

    def _output_list_item(self) -> None:
        for product in self.products:
            # 1.处理名称，控制字符在40（或规定长度），按照规则处理
            name = self._handle_product_name(product)
            # 2.处理价格
            price = self._handle_product_price(product)
            # 3. 输出
            self._output_line_item(name, price)

    def _get_total_price(self) -> float:
        return sum([product.price for product in self.products])

    def _get_sumerize(self, total_price: str) -> str:
        return f'Total: {total_price:>{self.price_length}}'

    def _output_sumerize(self) -> None:
        # 1.获取总价
        total_price = self._get_total_price()
        # 2.产生输出内容字符串
        sumerize_content = self._get_sumerize(total_price)
        # 3.输出
        total_length = self.name_list_length + self.price_length
        self._output(f'{sumerize_content:>{total_length}}')

    def report(self):
        # 1.输出头部
        self._output_header()
        # 2.输出列表主题
        self._output_list_item()
        # 3.输出统计部分
        self._output_sumerize()


def register_outputer():
    register('print', PrintOutputer)
    register('file', FileOutputer)
    register('combine', CombineOutputer)


def load_setting():
    with open('setting.json', 'r') as f:
        return json.load(f)


def main(outputer: Outputer):
    data = (
        {'name': 'Apple is a super good fruit',
         'price': 45.3},
        {'name': 'Pear is qjdl;asmdpwjasldsdsadsadasdjwsdadadsaddsadaaosdnlawioask',
         'price': 18.5},
        {'name': 'Orange is a super delicious fruit',
         'price': 23.3}
    )

    products = [Product(**item) for item in data]
    repoter = ProductReporter(products, outputer)
    repoter.report()


if __name__ == '__main__':
    register_outputer()
    outputer_setting = load_setting()
    outputer = create(outputer_setting['outputer'])
    main(outputer)

                            NAME                            PRICE
_________________________________________________________________
Apple is a super good fruit                                  45.3
-----------------------------------------------------------------
Pear is qjdl;asmdpwj..addsadaaosdnlawioask                   18.5
-----------------------------------------------------------------
Orange is a super delicious fruit                            23.3
-----------------------------------------------------------------
                                                     Total:  87.1
